In [1]:
# System level imports
import sys
import os

# ##### SET SYS PATH TO WHERE THE CODE IS. #####
# Note: Putting our code first in the sys path will make sure it gets picked up
llm_analyst_base_dir='/Users/dan/Code/Vibe_Coding/judging-app/backend'
sys.path.insert(0, llm_analyst_base_dir)

In [2]:
from app.schemas.user import UserCreate, UserRead, LoginRequest
from app.models.core_db import get_db
from app.services.auth_service import AuthService

In [15]:
#magic_link_token = 'zgWHM3gnGNK5NVEzzsm8V3gkqMYQF_D0EG6DZoptBoo'
db_gen = get_db()
db = next(db_gen)
service = AuthService(db)

user_verified = service.verify(magic_link_token)

In [13]:
# Send a magic link account must exist already
email = "dan2@example.com"
db_gen = get_db()
db = next(db_gen)
service = AuthService(db)

user_data = service.send_magic_link(email)
print("magic_link_token:", user_data.magic_link_token)
magic_link_token = user_data.magic_link_token

DEBUG:root:SMTP sent to: dan2@example.com


magic_link_token: fWcN96O_lYBQKnP-AIS2zOe0YKxJ8cBj3aCkviHegH8


In [4]:
# Register a new user
user_dict = {"first_name": "Dan", "last_name": "Higgins", "email": "dan2@example.com", "password": "password", "organization": "Vibe Coding"}
new_user = UserCreate.model_validate(user_dict)

# Manual way to get db session
db_gen = get_db()
db = next(db_gen)  # get the Session

service = AuthService(db)
user = service.register(new_user)
print(user)

DEBUG:root:SMTP sent to: dan2@example.com


id=1 first_name='Dan' last_name='Higgins' email='dan2@example.com' password='$argon2id$v=19$m=65536,t=3,p=4$GmMsRYjROuecE0JIiTEmRA$VViVRjrKMi5UDkrDtZtq0qAcip350OLLOSGxwKy7AJM' organization='Vibe Coding' magic_link_token='N32NADetO4PChdUanjivYJBucxt_qfYwQcDrCbtzPLo' magic_link_expires_at='2025-08-25T20:05:59.462941Z' last_login_at='2025-08-25T19:50:59.462953Z' is_verified=False registered_at=datetime.datetime(2025, 8, 25, 15, 50, 59, 430268) role='user'


In [16]:
# Check how long a user has been logged in for
db_gen = get_db()
db = next(db_gen)
service = AuthService(db)

minutes_logged_in = service.active_login_minutes("dan2@example.com")
print(f"{minutes_logged_in=}")

minutes_logged_in=0


In [9]:
# Sign in an existing user
db_gen = get_db()
db = next(db_gen)
service = AuthService(db)

user_read = service.signin("dan2@example.com", "password")
print(user_read)

id=1 first_name='Dan' last_name='Higgins' email='dan2@example.com' password='$argon2id$v=19$m=65536,t=3,p=4$GmMsRYjROuecE0JIiTEmRA$VViVRjrKMi5UDkrDtZtq0qAcip350OLLOSGxwKy7AJM' organization='Vibe Coding' magic_link_token='N32NADetO4PChdUanjivYJBucxt_qfYwQcDrCbtzPLo' magic_link_expires_at='2025-08-25T20:05:59.462941Z' last_login_at='2025-08-25T19:54:15.801470Z' is_verified=False registered_at=datetime.datetime(2025, 8, 25, 15, 50, 59, 430268) role='user'


In [ ]:
from datetime import datetime, timedelta, timezone

email="dphiggins@gmail.com"
db_gen = get_db()
db = next(db_gen)
service = AuthService(db)

user_data = service.send_magic_link(email)


expires_at = user_data.magic_link_expires_at

# fallback in case it's naive for some reason
if expires_at and expires_at.tzinfo is None:
    expires_at = expires_at.replace(tzinfo=timezone.utc)

now = datetime.now(timezone.utc)

print("DEBUG expires_at:", expires_at.isoformat() if expires_at else None)
print("DEBUG        now:", now.isoformat())
if expires_at:
    delta = expires_at - now
    seconds = int(delta.total_seconds())
    mins, secs = divmod(abs(seconds), 60)

    # Positive = still time left, Negative = expired already
    sign = "" if seconds >= 0 else "-"
    print(f"DEBUG Time before expires: {sign}{mins} min {secs} sec")
else:
    print("DEBUG expires_at is None")
            

In [ ]:
import requests

# API endpoint
url = "http://localhost:8000/api/v1/auth/verify"  # adjust if running on a different host/port

# Example token (replace with a real one from your app)
token = "IRf08SCIoeJFc2CLVE-oUXA9Ql8pIGa-s-n_wWMz_Vg"

# Request payload
payload = {"token": token}

# Send POST request
response = requests.post(url, json=payload)

# Print debug info
print("Status code:", response.status_code)
try:
    print("Response JSON:", response.json())
except Exception:
    print("Raw response:", response.text)